In [4]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen
import urllib
from dateutil.relativedelta import relativedelta

import gspread 
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

import hubspot

# Accessing Document with keys - not accessable via github
from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('notebook.cfg')

# All necessary keys
hubspot_arr_sync_apikey = parser.get('hubspot', 'ARR-Sync')
click_up_apikey = parser.get('click_up', 'api_key')
client_success_payload = parser.get('client_success', 'payload')
maxio_api_key = parser.get('maxio', 'api_key')
gc = gspread.service_account_from_dict(json.loads(parser.get('google', 'credentials')))

### Pull in Maxio information

In [5]:
def get_call(type):
    _df = pd.DataFrame()
    url = f'https://m12.saasoptics.com/coursekey/api/v1.0/{type}/'
    headers = {
    'Authorization': f'Token {maxio_api_key}'
    }
    has_more = True

    while has_more:
        response = requests.request("GET", url, headers=headers)
        _df = pd.concat([_df, pd.DataFrame(response.json()['results'])]).reset_index().drop('index', axis=1)

        if response.json()['next'] == None:
            has_more = False
        else:
            url = response.json()['next']

    _df = _df.dropna(how='all', axis=1)

    return _df

In [6]:
customers_raw_df = get_call('customers')
contracts_raw_df = get_call('contracts')
transactions_raw_df = get_call('transactions')
items_raw_df = get_call('items')

In [8]:
invoices_raw_df = get_call('invoices')

In [10]:
payments_raw_df = get_call('payments')

In [23]:
customers_raw_df.query("id == 4697")

,id,name,billing_profile,number,domain,auditentry,notes,is_active,qb_id,do_not_sync,unbalanced_revenue_exception,modified,sf_id,sf_opportunity_price_book_id,einvoicing_id,email,cc_email,escalation_email,default_email_from_so,default_enable_cc_payment,default_enable_ach_payment,autopay_enrollment,parent,default_theme,crm_id,text_field1,text_field2,industry,segment,market,c_business_unit
399,4697,Paul Mitchell The School - Shreveport,{'name': 'Paul Mitchell The School - Shrevepor...,4108795179,,"{'created': '2022-06-02T12:10:00.283389', 'cre...",,True,1011,False,False,2022-10-27T19:30:24.350552,,,None,,,,False,False,False,disabled,NaN,NaN,4108795179,None,None,None,None,None,None


In [25]:
customers_df = customers_raw_df.rename({'id':'customer_id', 'number':'hubspot_id'}, axis=1)
customers_df = customers_df.reindex(['customer_id', 'name', 'hubspot_id'], axis=1)

In [31]:
customers_df.head()

,customer_id,name,hubspot_id
0,4714,1.Enterprise,2022-1034
1,4384,2. Instructors,2022-704
2,4476,3. Investors,2022-796
3,4573,5T Beauty Company,7647293872
4,4481,ABILENE CHRISTIAN UNIVERSITY,2022-801


In [35]:
contracts_raw_df.query("id == 6567")

,id,number,billing_profile,notes,is_job,unbalanced_revenue_exception,email,sf_id,register,customer,created,created_by,created_by_name,modified,modified_by,modified_by_name,lead_source,c_account_executive,c_business_unit,c_close_date,c_segment
28,6567,2022-542,None,None,False,False,,,48,4697,2022-06-02T12:10:15.843724,None,QuickBooks,2022-06-02T12:10:15.843740,NaN,QuickBooks,None,None,None,None,None


In [36]:
contracts_df = contracts_raw_df.rename({'id':'contract_id', 'number':'deal_id', 'c_close_date':'deal_close_date'}, axis=1)
contracts_df = contracts_df.reindex(['contract_id', 'deal_id', 'deal_close_date'], axis=1)

In [37]:
contracts_df.head()

,contract_id,deal_id,deal_close_date
0,6535,2022-510,None
1,6536,2022-511,None
2,6537,2022-512,2022-06-29
3,6538,2022-513,None
4,6539,2022-514,None


In [20]:
payments_raw_df.query("number == '2022-5662'")

,id,lines,source_system,source_id,source_created,source_modified,date,local_amount,local_applied_amount,local_unapplied_amount,test,number,reference_number,description,sync,type,status,contract,customer,failure_code,failure_message,qb_payment_method,funding_name,foreign_exchange_rate,sync_to_chargify,qb_id,created,created_by_name,modified,modified_by,modified_by_name
3082,11637,"[{'id': 25437, 'invoice': 26175, 'local_amount...",qb,28586,2022-08-22T13:34:41,2022-08-22T13:34:41,2022-08-22,1937.60,1937.60,0.00,False,2022-5662,28586,,True,payment,succeeded,6567.0,4697,,,75.0,,1.000000,True,28586,2022-08-28T18:41:21.661642,QuickBooks,2022-08-28T18:41:21.661664,NaN,QuickBooks


In [86]:
### Pull out information from lines within the payments dataframe and create a new dataframe with the information
payments_df = pd.DataFrame()

for i in range(len(payments_raw_df)):
    for line in range(len(payments_raw_df['lines'][i])):
        line_info = payments_raw_df['lines'][i][line]
        line_info['payment_number'] = payments_raw_df['number'][i]
        line_info['payment_date'] = payments_raw_df['date'][i]
        line_info['total_paid'] = payments_raw_df['local_amount'][i]
        line_info['customer_id'] = payments_raw_df['customer'][i]
        line_df = pd.DataFrame([line_info])
        payments_df = pd.concat([payments_df, line_df], ignore_index=True)

payments_df = payments_df.rename({'id':'payment_id', 'invoice':'invoice_id'}, axis=1)
payments_df = payments_df.reindex(['customer_id', 'invoice_id', 'payment_id', 'payment_number', 'payment_date', 'total_paid'], axis=1)
payments_df.head()

,customer_id,invoice_id,payment_id,payment_number,payment_date,total_paid
0,4627,25688,22175,2022-2,2016-11-15,0.00
1,4627,25689,22177,2022-3,2017-02-17,0.00
2,4627,25690,22179,2022-4,2017-03-07,0.00
3,4627,25691,22181,2022-5,2017-04-13,0.00
4,4627,25692,22183,2022-6,2017-06-02,0.00


In [14]:
invoices_raw_df.query('number == "2022-10072"')

,id,auditentry,number,date,due_date,po_number,memo,other,to_be_printed,to_be_emailed,billing_addr1,billing_addr2,billing_addr3,billing_city,billing_state,billing_zip_code,billing_country,shipping_addr1,shipping_addr2,shipping_addr3,shipping_city,shipping_state,shipping_zip_code,shipping_country,exported_date,do_not_sync,ignore_date_when_syncing,type,qb_number,qb_txn_id,deleted_in_qb,sync_date,foreign_exchange_rate,subtotal,is_paid,applied_amount,balance,email_from_so,sf_id,contract,qb_payment_terms,ei_theme,committed,line_items,local_amount,home_amount,tax_lines,einvoicing_url,einvoicing_url_no_click_tracking
5,21624,"{'qb_created': '2022-12-01T21:31:07', 'qb_modi...",2022-10072,2022-12-01,2022-12-31,,,,False,False,1701 Old Minden Road,Suite 8,,Bossier City,LA,71111,United States,1701 Old Minden Road,Suite 8,,Bossier City,LA,71111,United States,2022-12-01T21:27:17.115170,False,False,Invoice,2022-10072,30651,False,2022-12-02T14:45:16.156920,1.000000,1651.20,False,0.00,1651.20,False,,6567,86.0,None,False,"[{'id': 50676, 'number': '2022-10081', 'home_a...",1651.20,1651.20,[],,


In [93]:
### Pull out information from lines within the invoices dataframe and create a new dataframe with the information
invoices_df = pd.DataFrame()

for i in range(len(invoices_raw_df)):
    for line in range(len(invoices_raw_df['line_items'][i])):
        line_info = invoices_raw_df['line_items'][i][line]
        line_info['invoice_number'] = invoices_raw_df['number'][i]
        line_info['due_date'] = invoices_raw_df['due_date'][i]
        line_info['total_invoiced'] = invoices_raw_df['local_amount'][i]
        line_info['is_paid'] = invoices_raw_df['is_paid'][i]
        line_df = pd.DataFrame([line_info])
        invoices_df = pd.concat([invoices_df, line_df], ignore_index=True)

invoices_df = invoices_df.rename({'id':'invoice_id'}, axis=1)
# invoices_df = invoices_df.reindex(['customer_id', 'invoice_id', 'payment_id', 'notes', 'payment_number', 'payment_date', 'total_paid'], axis=1)
invoices_df.head()

,invoice_id,number,home_amount,local_amount,quantity,notes,no_transaction_permitted,exported_date,qb_txn_line_id,sync_date,deleted_in_qb,qb_time_modified,sf_id,stripe_id,refund_of_stripe_id,recurly_id,invoice,item,transaction,qb_class,refund_of,external_id,created,created_by,created_by_name,modified,modified_by,modified_by_name,invoice_number,due_date,total_invoiced,is_paid
0,51288,2022-10693,1320.00,1320.00,0.0000000,Sync Attendance subscription billing from 11/0...,False,2022-10-27T19:31:45.395868,2022-10034-30601-1,2023-01-04T14:27:06.218671,False,2023-01-02T19:31:10,,None,None,None,21586,8699,61443,231,None,None,2022-10-02T21:52:37.410268,34,Nicole Bird,2023-01-04T14:27:08.463997,None,QuickBooks,2022-10034,2022-12-31,1171.50,True
1,52078,2022-11483,-148.50,-148.50,1.0000000,Index adjustment effective 11/01/2022,False,None,2022-10034-30601-2,2023-01-04T14:27:06.218671,False,2023-01-02T19:31:10,,None,None,None,21586,8707,61912,231,None,None,2022-11-30T15:54:10.341165,34,Nicole Bird,2023-01-04T14:27:08.482162,None,QuickBooks,2022-10034,2022-12-31,1171.50,True
2,51291,2022-10696,1320.00,1320.00,0.0000000,Sync Attendance subscription billing from 02/0...,False,2023-01-31T17:07:27.651377,2022-10035-31474-1,2023-02-07T12:06:50.059787,False,2023-02-01T15:51:35,,None,None,None,21587,8699,61443,231,None,None,2022-10-02T21:52:37.466968,34,Nicole Bird,2023-02-07T12:06:51.902968,None,QuickBooks,2022-10035,2023-03-03,1171.50,True
3,52083,2022-11488,-148.50,-148.50,1.0000000,Index adjustment effective 11/01/2022,False,2023-01-31T17:07:27.651377,2022-10035-31474-2,2023-02-07T12:06:50.059787,False,2023-02-01T15:51:35,,None,None,None,21587,8707,61912,231,None,None,2022-11-30T15:54:10.447886,34,Nicole Bird,2023-02-07T12:06:51.927583,None,QuickBooks,2022-10035,2023-03-03,1171.50,True
4,50673,2022-10078,1380.00,1380.00,1.0000000,This is your subscription fee for Sync Attenda...,False,2022-10-18T12:33:52.974054,2022-10069-28710-1,2022-09-19T10:53:21.398164,False,2022-09-01T19:34:54,,None,None,None,21621,8699,61695,231,None,None,2022-08-26T20:50:57.809054,34,Nicole Bird,2022-09-19T10:53:23.370691,None,QuickBooks,2022-10069,2022-10-01,1932.00,False


In [ ]:
invoices_df.reindex(['invoice_id', 'invoice_id', 'payment_id', 'notes', 'payment_number', 'payment_date', 'total_paid'], axis=1)

In [17]:
invoices_df = invoices_raw_df.rename({'number':'invoice_number', 'local_amount':'total_invoiced'}, axis=1)
invoices_df = invoices_df.reindex(['invoice_number', 'due_date', 'is_paid', 'total_invoiced'], axis=1)

In [33]:
invoices_df.head()

,invoice_number,date,due_date,is_paid,total
0,2022-10034,2022-11-01,2022-12-31,True,1171.50
1,2022-10035,2023-02-01,2023-03-03,True,1171.50
2,2022-10069,2022-09-01,2022-10-01,False,1932.00
3,2022-10070,2022-10-01,2022-10-31,False,1932.00
4,2022-10071,2022-11-01,2022-12-01,False,1931.20


In [16]:
invoices_raw_df['line_items'][5][0]

{'id': 50676,
 'number': '2022-10081',
 'home_amount': '1376.00',
 'local_amount': '1376.00',
 'quantity': '1.0000000',
 'notes': 'This is your subscription fee for Sync Attendance for the term starting 07/01/2022 and ending 12/31/2022. This invoice billing period is from 12/01/2022 through 12/31/2022.',
 'no_transaction_permitted': False,
 'exported_date': '2022-11-28T19:20:43.286816',
 'qb_txn_line_id': '2022-10072-30651-1',
 'sync_date': '2022-12-02T14:45:16.156920',
 'deleted_in_qb': False,
 'qb_time_modified': '2022-12-01T21:35:39',
 'sf_id': '',
 'stripe_id': None,
 'refund_of_stripe_id': None,
 'recurly_id': None,
 'invoice': 21624,
 'item': 8699,
 'transaction': 61695,
 'qb_class': 231,
 'refund_of': None,
 'external_id': None,
 'created': '2022-08-26T20:50:57.890531',
 'created_by': 34,
 'created_by_name': 'Nicole Bird',
 'modified': '2022-12-02T14:45:16.232944',
 'modified_by': None,
 'modified_by_name': 'QuickBooks'}